In [10]:
# import rainymotion library
from rainymotion import models, metrics

# import accompanying libraries
from collections import OrderedDict
import numpy as np
import h5py
import matplotlib.pyplot as plt

% matplotlib inline

In [2]:
data = h5py.File("../data/data.h5", mode="r")
eval_idx = np.load("../data/eval_dict.npy").item()

In [14]:
eval_idx = OrderedDict({key : eval_idx[key] for key in sorted(list(eval_idx.keys()))})

In [16]:
sorted(list(eval_idx.keys()))

['201605291205',
 '201605291210',
 '201605291215',
 '201605291220',
 '201605291225',
 '201605291230',
 '201605291235',
 '201605291240',
 '201605291245',
 '201605291250',
 '201605291255',
 '201605291300',
 '201605291305',
 '201605291310',
 '201605291315',
 '201605291320',
 '201605291325',
 '201605291330',
 '201605291335',
 '201605291340',
 '201605291345',
 '201605291350',
 '201605291355',
 '201605291400',
 '201605291405',
 '201605291410',
 '201605291415',
 '201605291420',
 '201605291425',
 '201605291430',
 '201605291435',
 '201605291440',
 '201605291445',
 '201605291450',
 '201605291455',
 '201605291500',
 '201605291505',
 '201605291510',
 '201605291515',
 '201605291520',
 '201605291525',
 '201605291530',
 '201605291535',
 '201605291540',
 '201605291545',
 '201605291550',
 '201605291555',
 '201605291600',
 '201605291605',
 '201605291610',
 '201605291615',
 '201605291620',
 '201605291625',
 '201605291630',
 '201605291635',
 '201605291640',
 '201605291645',
 '201605291650',
 '201605291655

In [18]:
results = h5py.File("../data/results.h5")

In [19]:
def ground_truth(data_instance, eval_instance, results_instance):
    
    results_instance.create_group("/GT/")
    
    for key in sorted(list(eval_instance.keys())):
               
        ground_truth = np.array([ data_instance[key][()] for key in eval_instance[key][1] ])
        
        results_instance["/GT/"].create_dataset(key, 
                                                data=ground_truth, 
                                                dtype="float16", 
                                                chunks=(12,100,100), 
                                                maxshape=(12,900,900), compression="gzip")

In [20]:
%%time
ground_truth(data, eval_idx, results)

201605291205
201605291210
201605291215
201605291220
201605291225
201605291230
201605291235
201605291240
201605291245
201605291250
201605291255
201605291300
201605291305
201605291310
201605291315
201605291320
201605291325
201605291330
201605291335
201605291340
201605291345
201605291350
201605291355
201605291400
201605291405
201605291410
201605291415
201605291420
201605291425
201605291430
201605291435
201605291440
201605291445
201605291450
201605291455
201605291500
201605291505
201605291510
201605291515
201605291520
201605291525
201605291530
201605291535
201605291540
201605291545
201605291550
201605291555
201605291600
201605291605
201605291610
201605291615
201605291620
201605291625
201605291630
201605291635
201605291640
201605291645
201605291650
201605291655
201605291700
201605291705
201605291710
201605291715
201605291720
201605291725
201605291730
201605291735
201605291740
201605291745
201605291750
201605291755
201605291800
201605291805
201605291810
201605291815
201605291820
201605291825

In [21]:
def persistence(data_instance, eval_instance, results_instance):
    
    results_instance.create_group("/Persistence/")
    
    for key in sorted(list(eval_instance.keys())):
        
        inputs = np.array([ data_instance[key][()] for key in eval_instance[key][1][-1:] ])
        
        model = models.EulerianPersistence()
        
        model.input_data = inputs
        
        nowcast = model.run()
        
        results_instance["/Persistence/"].create_dataset(key,
                                                         data=nowcast, 
                                                         dtype="float16",
                                                         chunks=(12,100,100),
                                                         maxshape=(12,900,900), 
                                                         compression="gzip")

In [22]:
%%time
persistence(data, eval_idx, results)

CPU times: user 56.7 s, sys: 696 ms, total: 57.4 s
Wall time: 57.5 s


In [ ]:
def optical_flow(data_instance, eval_instance, results_instance, model_name):
    